In [1]:
from pathlib import Path
import os
from pydub import AudioSegment

os.chdir('e:/code/python/WangXiang')

In [2]:
def pr_info(audio):
    print(f'时长（秒）：{len(audio) / 1000.0}')
    print(f'声道数（channels）：{audio.channels}')
    print(f'采样率（frame_rate）：{audio.frame_rate}')
    print(f'帧宽（sample_width）：{audio.sample_width} 字节')


In [3]:
audio = AudioSegment.from_file(Path('data/recording/audio/dailog.mp3'))
channels = audio.split_to_mono()
left_channel = channels[0]
right_channel = channels[1]

In [4]:
segment_length = 200  # 毫秒


def segment_audio(audio, segment_length):
    segments = []
    for start_ms in range(0, len(audio), segment_length):
        segment = audio[start_ms : start_ms + segment_length]
        segments.append(segment)
    return segments


left_segments = segment_audio(left_channel, segment_length)
right_segments = segment_audio(right_channel, segment_length)

n = len(left_segments)


In [5]:
flags = []
for idx in range(n):
    if left_segments[idx].rms > right_segments[idx].rms:
        flags.append(0)
    else:
        flags.append(1)

In [6]:
def modify_segments(arr, min_sec):
    min_count = int(min_sec / (segment_length / 1000.0))
    print(f'最小连续段数：{min_count}')
    i = 0
    while i < len(arr):
        count = 1
        while i + 1 < len(arr) and arr[i] == arr[i + 1]:
            count += 1
            i += 1
        if count < min_count:
            for j in range(i - count + 1, i + 1):
                arr[j] = 1 - arr[i]

        i += 1
    return arr


# arr = [0, 0, 0,0,0,0,0,0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# 最短持续的秒数 ，2
modified_flags = modify_segments(flags, 1)

print(modified_flags)

最小连续段数：5
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [7]:
for idx in range(n):
    duration = len(left_segments[idx])
    if flags[idx] == 1:
        left_segments[idx] = AudioSegment.silent(duration=duration, frame_rate=8000)
    else:
        right_segments[idx] = AudioSegment.silent(duration=duration, frame_rate=8000)

In [8]:
combined_left_audio = AudioSegment.empty()
for seg in left_segments:
    combined_left_audio += seg

combined_right_audio = AudioSegment.empty()
for seg in right_segments:
    combined_right_audio += seg


In [ ]:
combined_audio = AudioSegment.from_mono_audiosegments(combined_left_audio, combined_right_audio)
# combined_audio.export('合成.mp3', format='mp3')

<_io.BufferedRandom name='合成.mp3'>